<a href="https://colab.research.google.com/github/Attabeezy/sequential-crm-for-dce/blob/main/ctgan_syn_data_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.6/196.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
"""
CTGAN synthetic data generator using SDV's CTGANSynthesizer and metadata API
Updated to use sdv >= 1.0 (sdv.single_table.CTGANSynthesizer)

Usage:
    pip install pandas scikit-learn sdv openpyxl

    python ctgan_sdv_generator_new.py --input MomoStatementReport.xlsx --sheet Sheet1 \
        --n-samples 10000 --epochs 300 --output synthetic.csv

Notes:
- This script uses SDV's SingleTableMetadata to infer column types. You may still
  override columns if autodetection isn't perfect.
- The script generates independent rows (CTGAN). For sequence-aware generation,
  use a sequence model (not included here).
"""

import argparse
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

## Helpers

In [ ]:
def detect_datetime_columns(df, sample_n=1000):
    dt_cols = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.datetime64):
            dt_cols.append(col)
            continue
        if df[col].dtype == object:
            # Ensure sample size does not exceed the number of available non-null elements
            non_null_count = len(df[col].dropna())
            actual_sample_size = min(non_null_count, sample_n)
            if actual_sample_size == 0: # Skip if no non-null values to sample
                continue
            sample = df[col].dropna().astype(str).sample(actual_sample_size, random_state=1)
            parsed = sample.apply(lambda x: pd.to_datetime(x, errors='coerce')).notna().mean()
            if parsed > 0.9:
                dt_cols.append(col)
    return dt_cols


def group_rare_levels(s, top_k=50, min_freq=0.01):
    freqs = s.value_counts()
    n = len(s)
    keep = set(freqs.head(top_k).index)
    keep2 = set(freqs[freqs / n >= min_freq].index)
    final_keep = keep.union(keep2)
    return s.where(s.isin(final_keep), other='__OTHER__')

## Preprocessing

In [ ]:
def preprocess_dataframe(df):
    df = df.copy()

    # detect and convert datetime columns
    dt_cols = detect_datetime_columns(df)
    created_cols = []
    for c in dt_cols:
        df[c] = pd.to_datetime(df[c], errors='coerce')
        # Use .astype('int64') instead of .view('int64') to avoid FutureWarning
        df[c + '__unix'] = df[c].astype('int64') // 10**9
        df[c + '__hour'] = df[c].dt.hour.astype('Int64')
        df[c + '__weekday'] = df[c].dt.dayofweek.astype('Int64')
        df[c + '__month'] = df[c].dt.month.astype('Int64')
        created_cols += [c + '__unix', c + '__hour', c + '__weekday', c + '__month']
        df = df.drop(columns=[c])

    # auto-detect categorical columns: object dtype or small-cardinality integer
    candidate_cats = []
    n = len(df)
    # Use pd.api.types.is_integer_dtype for robustness with pandas nullable integer types
    import pandas.api.types as ptypes
    for col in df.columns:
        if df[col].dtype == object:
            candidate_cats.append(col)
        elif ptypes.is_integer_dtype(df[col].dtype): # Changed from np.issubdtype
            nunique = df[col].nunique()
            if nunique <= 50 or (nunique / max(1, n)) <= 0.05:
                candidate_cats.append(col)

    # convert candidate cats to string and group rare levels
    for c in candidate_cats:
        df[c] = df[c].astype(str)
        df[c] = group_rare_levels(df[c])

    # Impute missing values: categorical -> '__NA__', numeric -> median
    categorical_columns = [c for c in candidate_cats if c in df.columns]
    for c in df.columns:
        if c in categorical_columns:
            df[c] = df[c].fillna('__NA__')
        else:
            if df[c].isna().any():
                try:
                    df[c] = df[c].fillna(df[c].median())
                except Exception:
                    df[c] = df[c].fillna(0)

    return df, created_cols, categorical_columns

## Metadata & CTGAN

In [ ]:
def build_metadata(df, categorical_columns):
    """Build SDV SingleTableMetadata and annotate detected categorical columns.
    Returns metadata object.
    """
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(df)

    # Force categorical columns to be discrete/string in metadata
    for c in categorical_columns:
        if c in metadata.get_columns():
            metadata.update_column(c, sdtype='categorical')
    return metadata


def train_ctgan_with_metadata(df, metadata, epochs=300):
    """Train CTGANSynthesizer using provided metadata."""
    model = CTGANSynthesizer(metadata=metadata, epochs=epochs)
    model.fit(df)
    return model

## Sampling & Evaluation

In [ ]:
def sample_ctgan(model, n_samples=10000, conditions=None):
    if conditions:
        # SDV supports conditional sampling via sample_conditions in some versions.
        # The API expects a list of condition dicts describing desired column values.
        # Try direct call; fallback to rejection sampling.
        try:
            # example: conditions = [{'column': 'tx_type', 'value': 'withdrawal'}]
            return model.sample_conditions(conditions)
        except Exception:
            pool = model.sample(n_samples * 4)
            for k, v in conditions.items():
                pool = pool[pool[k] == v]
            return pool.head(n_samples).reset_index(drop=True)
    return model.sample(num_rows=n_samples)


def quick_tstr(synth, holdout, label_col='label'):
    if label_col not in synth.columns or label_col not in holdout.columns:
        print('Label missing; skipping TSTR')
        return None
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score

    Xs = pd.get_dummies(synth.drop(columns=[label_col]))
    ys = synth[label_col].astype(int)
    Xr = pd.get_dummies(holdout.drop(columns=[label_col]))
    yr = holdout[label_col].astype(int)

    for c in Xs.columns.difference(Xr.columns):
        Xr[c] = 0
    for c in Xr.columns.difference(Xs.columns):
        Xs[c] = 0
    Xs = Xs[Xr.columns]

    clf = RandomForestClassifier(n_estimators=200, random_state=0)
    clf.fit(Xs, ys)
    preds = clf.predict_proba(Xr)[:,1]
    auc = roc_auc_score(yr, preds)
    print('TSTR AUC:', auc)
    return auc

## CLI

In [ ]:
def main(args):
    # Load
    if args.sheet is None:
        df = pd.read_excel(args.input)
    else:
        df = pd.read_excel(args.input, sheet_name=args.sheet)
    print('Loaded', df.shape)

    # Preprocess
    proc, created_cols, cat_cols = preprocess_dataframe(df)
    print('Created time cols:', created_cols)
    print('Detected categorical cols:', cat_cols)

    # Split for holdout
    train_df, holdout_df = train_test_split(proc, test_size=0.2, random_state=42)

    # Build metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(train_df)
    for c in cat_cols:
        try:
            metadata.update_column(c, sdtype='categorical')
        except Exception:
            pass

    # Train CTGAN
    print('Training CTGAN (this may take a few minutes)...')
    model = CTGANSynthesizer(metadata=metadata, epochs=args.epochs)
    model.fit(train_df)

    # Save model via SDV API
    try:
        model.save(args.model_out)
        print('Model saved to', args.model_out)
    except Exception:
        # fallback: pickle
        import pickle
        with open(args.model_out, 'wb') as f:
            pickle.dump(model, f)
        print('Model pickled to', args.model_out)

    # Sample
    synth = sample_ctgan(model, n_samples=args.n_samples)
    synth.to_csv(args.output, index=False)
    print('Synthetic saved to', args.output)

    # Optional TSTR
    if args.label_col:
        quick_tstr(synth, holdout_df, label_col=args.label_col)


if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('--input', required=True)
    parser.add_argument('--sheet', default=None)
    parser.add_argument('--n-samples', type=int, default=10000)
    parser.add_argument('--epochs', type=int, default=300)
    parser.add_argument('--output', default='synthetic.csv')
    parser.add_argument('--model-out', default='ctgan_sdv_new.pkl')
    parser.add_argument('--label-col', default=None)

    # For Colab execution, we will create a dummy args object
    # You need to replace 'YOUR_INPUT_FILE.xlsx' with the actual path to your file
    class DummyArgs:
        def __init__(self):
            self.input = 'MomoStatementReport.xlsx' # <--- REPLACE WITH YOUR FILE PATH
            self.sheet = None
            self.n_samples = 10000
            self.epochs = 300
            self.output = 'synthetic.csv'
            self.model_out = 'ctgan_sdv_new.pkl'
            self.label_col = None

    args_for_colab = DummyArgs()
    main(args_for_colab) # Call the main function with the dummy arguments

Loaded (105, 16)
Created time cols: ['TRANSACTION DATE__unix', 'TRANSACTION DATE__hour', 'TRANSACTION DATE__weekday', 'TRANSACTION DATE__month']
Detected categorical cols: ['FROM ACCT', 'FROM NAME', 'FROM NO.', 'TRANS. TYPE', 'TO NO.', 'TO NAME', 'TO ACCT', 'REF', 'OVA', 'TRANSACTION DATE__unix', 'TRANSACTION DATE__hour', 'TRANSACTION DATE__weekday', 'TRANSACTION DATE__month']
Training CTGAN (this may take a few minutes)...


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/ctgan/synthesizers/_utils.py:16: FutureWarning: `cuda` parameter is deprecated and will be removed in a future release. Please use `enable_gpu` instead.
  warnings.warn(


Model saved to ctgan_sdv_new.pkl
Synthetic saved to synthetic.csv



def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--input', required=True)
    parser.add_argument('--sheet', default=None)
    parser.add_argument('--n-samples', type=int, default=10000)
    parser.add_argument('--epochs', type=int, default=300)
    parser.add_argument('--output', default='synthetic.csv')
    parser.add_argument('--model-out', default='ctgan_sdv_new.pkl')
    parser.add_argument('--label-col', default=None)
    args = parser.parse_args()

    # Load
    if args.sheet is None:
        df = pd.read_excel(args.input)
    else:
        df = pd.read_excel(args.input, sheet_name=args.sheet)
    print('Loaded', df.shape)

    # Preprocess
    proc, created_cols, cat_cols = preprocess_dataframe(df)
    print('Created time cols:', created_cols)
    print('Detected categorical cols:', cat_cols)

    # Split for holdout
    train_df, holdout_df = train_test_split(proc, test_size=0.2, random_state=42)

    # Build metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(train_df)
    for c in cat_cols:
        try:
            metadata.update_column(c, sdtype='categorical')
        except Exception:
            pass

    # Train CTGAN
    print('Training CTGAN (this may take a few minutes)...')
    model = CTGANSynthesizer(metadata=metadata, epochs=args.epochs)
    model.fit(train_df)

    # Save model via SDV API
    try:
        model.save(args.model_out)
        print('Model saved to', args.model_out)
    except Exception:
        # fallback: pickle
        import pickle
        with open(args.model_out, 'wb') as f:
            pickle.dump(model, f)
        print('Model pickled to', args.model_out)

    # Sample
    synth = sample_ctgan(model, n_samples=args.n_samples)
    synth.to_csv(args.output, index=False)
    print('Synthetic saved to', args.output)

    # Optional TSTR
    if args.label_col:
        quick_tstr(synth, holdout_df, label_col=args.label_col)


if __name__ == '__main__':
    main()
